In [1]:
import pandas as pd
cal = pd.read_csv('/data/linq/.qlib/qlib_data/cn_data/calendars/day.txt')
delta = 0
TODAY = str(cal.iloc[-1-delta,0])
YESTERDAY = str(cal.iloc[-2-delta,0])
# TODAY = str(cal.iloc[-2,0])
# YESTERDAY = str(cal.iloc[-3,0])

BT_START = '2024-11-01'

print(BT_START)
print(YESTERDAY)
print(TODAY)

2024-11-01
2025-04-02
2025-04-03


In [2]:
# BT_START = '2024-12-01'
# TODAY = '2024-12-09'

test_split = (TODAY, TODAY)
bt_split = (BT_START, YESTERDAY)
pos_txt = f'''\
date {TODAY}
cash 721.00
SZ000001 2100
SH600570 800
SZ000661 200
SH600115 4800
SZ002459 1300
SZ002129 2000
SZ000002 2300
SH600372 1400
SH600188 1100
'''

In [3]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[3855165:MainThread](2025-04-04 05:51:08,642) INFO - qlib.Initialization - [config.py:416] - default_conf: client.


[3855165:MainThread](2025-04-04 05:51:08,645) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.


[3855165:MainThread](2025-04-04 05:51:08,646) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


Record position history

In [4]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
# print(pos_history)
# save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
hold_lst = pos.get_stock_list()
print(dt.date())
print(account)
print(norm_pos)
print(hold_lst)

2025-04-03
{'SZ000001': {'amount': 2100}, 'SH600570': {'amount': 800}, 'SZ000661': {'amount': 200}, 'SH600115': {'amount': 4800}, 'SZ002459': {'amount': 1300}, 'SZ002129': {'amount': 2000}, 'SZ000002': {'amount': 2300}, 'SH600372': {'amount': 1400}, 'SH600188': {'amount': 1100}, 'cash': 721.0}
{'_settle_type': 'None', 'position': {'SZ000661': {'amount': 2163, 'price': 9.420173645019531}, 'SH600372': {'amount': 0, 'price': inf}, 'SH600188': {'amount': 6609, 'price': 2.1983649730682373}, 'SZ000001': {'amount': 13990, 'price': 1.702152132987976}, 'SZ002129': {'amount': 4184, 'price': 4.25372838973999}, 'SH600115': {'amount': 28241, 'price': 0.6407620906829834}, 'SZ000002': {'amount': 14867, 'price': 1.104588508605957}, 'SZ002459': {'amount': 17480, 'price': 0.8307113647460938}, 'SH600570': {'amount': 3727, 'price': 5.914237976074219}, 'cash': 721.0, 'now_account_value': nan}, 'init_cash': 721.0}
['SZ000661', 'SH600372', 'SH600188', 'SZ000001', 'SZ002129', 'SH600115', 'SZ000002', 'SZ002459

In [5]:

pos_history = load_position_history()
hold_day = {}
_norm_pos = norm_pos.__dict__['position']
print(_norm_pos)
dt_lst = sorted(list(pos_history.keys()))[::-1]
for k, v in _norm_pos.items():
    if not isinstance(v, dict):
        continue
    cnt = 0
    for dt in dt_lst:
        if str(dt.date())==TODAY:
            # print(dt.date(), TODAY)
            continue
        tmp_pos = pos_history[dt]
        if k not in tmp_pos.get_stock_list():
            break
        cnt += 1
    # print(k, cnt)
    hold_day[k] = cnt
print(hold_day)

{'SZ000661': {'amount': 2163, 'price': 9.420173645019531}, 'SH600372': {'amount': 0, 'price': inf}, 'SH600188': {'amount': 6609, 'price': 2.1983649730682373}, 'SZ000001': {'amount': 13990, 'price': 1.702152132987976}, 'SZ002129': {'amount': 4184, 'price': 4.25372838973999}, 'SH600115': {'amount': 28241, 'price': 0.6407620906829834}, 'SZ000002': {'amount': 14867, 'price': 1.104588508605957}, 'SZ002459': {'amount': 17480, 'price': 0.8307113647460938}, 'SH600570': {'amount': 3727, 'price': 5.914237976074219}, 'cash': 721.0, 'now_account_value': nan}
{'SZ000661': 0, 'SH600372': 0, 'SH600188': 0, 'SZ000001': 0, 'SZ002129': 0, 'SH600115': 0, 'SZ000002': 0, 'SZ002459': 0, 'SH600570': 0}


In [6]:
def simp_eval(res_df, k, t=None, day=None):
    # print(res_df)
    if day:
        date = str(cal.iloc[-day-1,0])
        res_df = res_df[res_df.index.get_level_values('datetime') >= date]
    # Step 1: Group by multiple indices
    grouped = res_df.groupby(['datetime'])

    # Step 2: Sort by 'value' within each group in descending order
    sorted_df = grouped.apply(lambda x: x.sort_values(by='score', ascending=False))
    sorted_df['win'] = sorted_df['label'] * sorted_df['score'] > 0
    # Step 3: Select the top 5 rows for each group
    # top_k_per_group = sorted_df.groupby(['datetime']).head(k)
    top_k_per_group = sorted_df.groupby(['datetime']).apply(lambda x: x.nlargest(k, 'score'))#.reset_index(level=1, drop=1)
    if t:
        # top_k_per_group = top_k_per_group.tail(t)
        top_k_per_group = top_k_per_group.groupby(['datetime']).apply(lambda x: x.nsmallest(t, 'score'))
    # print(top_k_per_group)
    top_k_ic = top_k_per_group['label'].corr(top_k_per_group['score'])
    s, n = top_k_per_group['win'].sum(), len(top_k_per_group)
    # print(s, n)
    top_k_win = 1.0 * top_k_per_group['win'].sum() / len(top_k_per_group)
    return top_k_ic, top_k_win 

def run_model(model, path='/home/linq/finance/qniverse/log'):
    res_df = pd.read_csv(f'{path}/{model}_backtest_return_{TODAY}.csv', index_col=['datetime','instrument'])

    def eval_day(day=None):
        top5ic, top5win = simp_eval(res_df, 5, day=day)
        top10ic, top10win = simp_eval(res_df, 10, 5, day=day)
        # top15ic, top15win = simp_eval(res_df, 15, 5, day=day)
        top20ic, top20win = simp_eval(res_df, 20, 10, day=day)
        # top25ic, top25win = simp_eval(res_df, 25, 5, day=day)
        top30ic, top30win = simp_eval(res_df, 30, 10, day=day)
        top40ic, top40win = simp_eval(res_df, 40, 10, day=day)
        top50ic, top50win = simp_eval(res_df, 50, 10, day=day)
        top60ic, top60win = simp_eval(res_df, 60, 10, day=day)
        top70ic, top70win = simp_eval(res_df, 70, 10, day=day)
        top80ic, top80win = simp_eval(res_df, 80, 10, day=day)        
        top90ic, top90win = simp_eval(res_df, 90, 10, day=day)
        top100ic, top100win = simp_eval(res_df, 100, 10, day=day)
        top110ic, top110win = simp_eval(res_df, 110, 10, day=day)
        top120ic, top120win = simp_eval(res_df, 120, 10, day=day)
        top130ic, top130win = simp_eval(res_df, 130, 10, day=day)
        top140ic, top140win = simp_eval(res_df, 140, 10, day=day)
        top150ic, top150win = simp_eval(res_df, 150, 10, day=day)
        topic, topwin = simp_eval(res_df, 50, day=day)
        # return [topic, topwin, top5ic, top5win, top10ic, top10win, top15ic, top15win, top20ic, top20win, top25ic, top25win, top30ic, top30win]
        return [topwin, top5win, top10win, top20win, top30win, top40win, top50win, top60win, top70win, top80win, top90win, top100win, top110win, top120win, top130win, top140win, top150win], [topic, top5ic, top10ic, top20ic, top30ic, top40ic, top50ic, top60ic, top70ic, top80ic, top90ic, top100ic, top110ic, top120ic, top130ic, top140ic, top150ic]
    day1metrics = eval_day(2)
    day5metrics = eval_day(6)
    day10metrics = eval_day(11)
    day20metrics = eval_day(21)
    
    pred_df = pd.read_csv(f'{path}/{model}_tomorrow_return_{TODAY}.csv', index_col=['datetime','instrument'])

    return day1metrics, day5metrics, day10metrics, day20metrics, pred_df, res_df

# Run

In [7]:
info_win_dict1, info_ic_dict1 = {}, {}
info_win_dict5, info_ic_dict5 = {}, {}
info_win_dict10, info_ic_dict10 = {}, {}
info_win_dict20, info_ic_dict20 = {}, {}
pred_lst = []

name_lst =  ['MENSEMBLE', 'NENSEMBLE', 'DENSEMBLE']
for model in name_lst:
    NAME = model
    print(model)
    day1metrics, day5metrics, day10metrics, day20metrics, pred_df, res_df = run_model(model)
    info_win_dict1[NAME], info_ic_dict1[NAME] = day1metrics
    info_win_dict5[NAME], info_ic_dict5[NAME] = day5metrics
    info_win_dict10[NAME], info_ic_dict10[NAME] = day10metrics
    info_win_dict20[NAME], info_ic_dict20[NAME] = day20metrics
    # for k in [1, 5, 10, 20]:
    #     eval(f'    info_win_dict{k}[NAME], info_ic_dict{k}[NAME] = day{k}metrics')
    _df = pred_df.loc[:,'score']
    _df.name = model
    # print(_df)
    pred_lst.append(_df)

MENSEMBLE


NENSEMBLE


DENSEMBLE


In [8]:
pred_df = pd.concat(pred_lst, axis=1)

# pred_df = pred_df.loc[:, ~pred_df.columns.duplicated()]
_pred_df = pred_df = (pred_df) / pred_df.std()
pred_df

MENSEMBLE  NENSEMBLE  DENSEMBLE
datetime   instrument                                 
2025-04-03 SH600933     3.361347   1.457650   3.310201
           SZ002265     3.213873   1.136651   3.425730
           SZ002681     3.039448   0.973944   3.142008
           SZ000034     2.694723   0.687998   2.982227
           SH603766     2.603594  -1.035408   2.747664
...                          ...        ...        ...
           SH603693    -4.685299  -2.682722  -4.818176
           SH603985    -4.787240  -2.261395  -5.016702
           SH603938    -5.668655  -2.788024  -5.895942
           SH600610    -6.190278  -0.905502  -7.017935
           SZ002549    -6.800913  -3.491829  -7.266439

[915 rows x 3 columns]

In [9]:

pred_df['mean'] = _pred_df.mean(axis=1)
pred_df['std'] = _pred_df.std(axis=1)
pred_df['score'] = pd.concat([_pred_df[n]*info_win_dict5[n][0]/_pred_df[n].std() for n in name_lst], axis=1).sum(axis=1)
# pred_df

In [10]:
rank_df = pred_df.droplevel(0).sort_values(by='score', ascending=False)
fields = ['$close/$factor']
_df = D.features(rank_df.index.values, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['close']
_df = _df.droplevel(1)
rank_df = pd.concat([rank_df,_df],axis=1,join='inner')
report_top = rank_df

In [11]:
hold_lst

['SZ000661',
 'SH600372',
 'SH600188',
 'SZ000001',
 'SZ002129',
 'SH600115',
 'SZ000002',
 'SZ002459',
 'SH600570']

In [12]:
pos_rank = rank_df.sort_values(by='score', ascending=True)
pos_rank = pos_rank[pos_rank.index.isin(hold_lst)]
hd = [hold_day[s] for s in pos_rank.index.values]
pos_rank['HD'] = hd
report_pos = pos_rank

In [13]:
color_lst = [n for n in report_pos.columns if n not in ['close', 'HD']]
# color_lst

In [14]:
sty_report_pos = report_pos.style.background_gradient(cmap='Blues_r', subset=color_lst)
sty_report_pos

,MENSEMBLE,NENSEMBLE,DENSEMBLE,mean,std,score,close,HD
instrument,,,,,,,,
SZ000002,-1.292511,-1.671886,-0.955192,-1.306529,0.292757,-2.048492,7.140000,0
SZ000661,-0.720946,-1.846867,-0.493671,-1.020495,0.591654,-1.622623,101.919991,0
SZ000001,-1.060484,-1.096692,-0.871576,-1.009584,0.098700,-1.574900,11.340001,0
SH600372,-0.171478,-2.512179,0.092297,-0.863786,1.170553,-1.412871,nan,0
SH600188,-0.119521,-0.160267,-0.145041,-0.141610,0.016810,-0.221259,13.209999,0
SZ002129,-0.038104,-0.211982,-0.067128,-0.105738,0.076055,-0.169074,8.900001,0
SH600115,0.220936,-0.430188,0.485833,0.092194,0.384885,0.122190,3.770000,0
SH600570,0.706514,0.362804,0.766916,0.612078,0.177980,0.942516,27.559999,0
SZ002459,2.202888,1.661371,2.212097,2.025452,0.257472,3.137799,11.170000,0


In [15]:
sty_report_top0 = report_top.head(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top1 = report_top.head(20).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top2 = report_top.head(30).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top3 = report_top.head(40).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top4 = report_top.head(50).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top5 = report_top.head(60).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top6 = report_top.head(70).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top7 = report_top.head(80).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top8 = report_top.head(90).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top9 = report_top.head(100).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top10 = report_top.head(110).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top11 = report_top.head(120).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top12 = report_top.head(130).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top13 = report_top.head(140).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top14 = report_top.head(150).tail(10).style.background_gradient(axis=0, subset=color_lst)

In [16]:
sty_report_top0

,MENSEMBLE,NENSEMBLE,DENSEMBLE,mean,std,score,close
instrument,,,,,,,
SH600933,3.361347,1.457650,3.310201,2.709733,0.885602,4.165895,16.049999
SZ002265,3.213873,1.136651,3.425730,2.592085,1.032775,3.974722,20.690001
SZ002681,3.039448,0.973944,3.142008,2.385133,0.998740,3.654229,7.230000
SZ000034,2.694723,0.687998,2.982227,2.121649,1.020517,3.243445,40.389999
SH603398,2.464326,1.085814,2.729828,2.093323,0.720614,3.217069,12.560000
SZ002459,2.202888,1.661371,2.212097,2.025452,0.257472,3.137799,11.170000
SH601689,2.285238,1.302455,2.356157,1.981283,0.480877,3.056032,56.609997
SZ002837,2.162687,1.457794,2.175570,1.932017,0.335367,2.987768,37.410000
SZ002123,2.274160,0.922369,2.633870,1.943466,0.736807,2.983383,14.270000


In [17]:
sty_report_top1

,MENSEMBLE,NENSEMBLE,DENSEMBLE,mean,std,score,close
instrument,,,,,,,
SH603583,2.083270,1.188064,2.103773,1.791702,0.426919,2.763966,38.700001
SZ002920,2.244690,0.676793,2.466456,1.795980,0.796546,2.749479,106.330002
SZ002139,2.200896,0.938534,2.212747,1.784059,0.597896,2.741975,14.280001
SZ002993,2.240518,0.818119,2.302666,1.787101,0.685643,2.741649,37.879997
SZ002163,2.140012,0.878165,2.173940,1.730705,0.602996,2.658666,7.350000
SZ002130,2.111268,0.929760,2.121452,1.720826,0.559384,2.645809,19.810001
SZ000555,2.002070,0.930657,2.205055,1.712594,0.559088,2.633696,11.410001
SH601865,1.942623,1.238258,1.930110,1.703664,0.329131,2.632624,17.500000
SZ002048,2.105117,0.611533,2.415680,1.710777,0.787555,2.617825,14.830000


In [18]:
sty_report_top2

,MENSEMBLE,NENSEMBLE,DENSEMBLE,mean,std,score,close
instrument,,,,,,,
SH601137,1.914513,0.937314,2.012586,1.621471,0.485426,2.495746,19.270000
SH603296,1.834296,1.038246,1.937017,1.603186,0.401668,2.472259,71.879997
SZ000528,2.206723,0.340958,2.305334,1.617672,0.903670,2.465143,11.270000
SH603328,2.124445,0.538727,2.117587,1.593586,0.745903,2.436719,9.480000
SH603915,1.887903,0.758085,2.101511,1.582500,0.589436,2.429430,16.330000
SZ002354,2.003717,0.474668,2.267189,1.581858,0.790256,2.416732,6.090000
SZ002698,1.864163,0.692999,2.075051,1.544071,0.607926,2.368494,17.300001
SZ000923,2.351169,-0.351393,2.645139,1.548305,1.348640,2.331531,15.119999
SH600811,1.881154,0.634350,2.040832,1.518779,0.628774,2.327647,0.470000


In [19]:
import dataframe_image as dfi

infodf = pd.DataFrame({'label': ['Model update date', 'Prediction generation date'],
                       'date': ['2024-12-11', TODAY]})

# dfi.export(infodf, './tmp/ensemble_all_info.png',table_conversion='matplotlib')

# dfi.export(sty_report_info5, './tmp/ensemble_all_performance5.png',table_conversion='matplotlib')
# dfi.export(sty_report_info10, './tmp/ensemble_all_performance10.png',table_conversion='matplotlib')
# dfi.export(sty_report_info20, './tmp/ensemble_all_performance20.png',table_conversion='matplotlib')
dfi.export(sty_report_top0, './tmp/ensemble_all_top0.png',table_conversion='matplotlib')
dfi.export(sty_report_top1, './tmp/ensemble_all_top1.png',table_conversion='matplotlib')
dfi.export(sty_report_top2, './tmp/ensemble_all_top2.png',table_conversion='matplotlib')
dfi.export(sty_report_top3, './tmp/ensemble_all_top3.png',table_conversion='matplotlib')
dfi.export(sty_report_top4, './tmp/ensemble_all_top4.png',table_conversion='matplotlib')
dfi.export(sty_report_top5, './tmp/ensemble_all_top5.png',table_conversion='matplotlib')
# dfi.export(sty_report_top6, './tmp/ensemble_all_top6.png',table_conversion='matplotlib')
# dfi.export(sty_report_top7, './tmp/ensemble_all_top7.png',table_conversion='matplotlib')
# dfi.export(sty_report_top8, './tmp/ensemble_all_top8.png',table_conversion='matplotlib')
# dfi.export(sty_report_top9, './tmp/ensemble_all_top9.png',table_conversion='matplotlib')
# dfi.export(sty_report_top10, './tmp/ensemble_all_top10.png',table_conversion='matplotlib')
# dfi.export(sty_report_top11, './tmp/ensemble_all_top11.png',table_conversion='matplotlib')
# dfi.export(sty_report_top12, './tmp/ensemble_all_top12.png',table_conversion='matplotlib')
# dfi.export(sty_report_top13, './tmp/ensemble_all_top13.png',table_conversion='matplotlib')
# dfi.export(sty_report_top14, './tmp/ensemble_all_top14.png',table_conversion='matplotlib')
# dfi.export(sty_report_pos, './tmp/ensemble_all_watch.png',table_conversion='matplotlib')

In [20]:

for k in [1,5,10,20]:
    sty_report_win_info = eval(f"pd.DataFrame(info_win_dict{k}, index=['TOP50WIN', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120-130', '130-140', '140-150'])").style.background_gradient(axis=None)
    sty_report_ic_info = eval(f"pd.DataFrame(info_ic_dict{k}, index=['TOP50IC', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120-130', '130-140', '140-150'])").style.background_gradient(axis=None)
    dfi.export(sty_report_win_info, f'./tmp/ensemble_all_win{k}.png',table_conversion='matplotlib')
    dfi.export(sty_report_ic_info, f'./tmp/ensemble_all_ic{k}.png',table_conversion='matplotlib')

In [21]:
dt_lst = sorted(list(pos_history.keys()))
cash, sv, tot = [], [], []
for dt in dt_lst:
    pos: Position = pos_history[dt]
    cash.append(pos.get_cash())
    sv.append(pos.calculate_stock_value())
    tot.append(pos.calculate_value())
res = pd.DataFrame({'Cash':cash, 'Stock Value':sv, 'Total Value':tot}, index=[d.date() for d in dt_lst])
res['Balance'] = res['Total Value'] - res['Total Value'].shift(1) 
res['Return'] = res['Balance'] / res['Total Value']
res

,Cash,Stock Value,Total Value,Balance,Return
2025-01-27,231890.0,125357.470266,357247.470266,NaN,NaN
2025-02-05,208683.0,146512.893952,355195.893952,-2051.576315,-0.005776
2025-02-06,209094.0,197512.050590,406606.050590,51410.156638,0.126437
2025-02-07,213032.0,189548.125394,402580.125394,-4025.925196,-0.010000
2025-02-10,217626.0,198247.362466,415873.362466,13293.237073,0.031965
2025-02-11,216700.0,205307.446995,422007.446995,6134.084529,0.014535
2025-02-12,220518.0,197773.868888,418291.868888,-3715.578108,-0.008883
2025-02-13,222893.0,191062.528317,413955.528317,-4336.340571,-0.010475
2025-02-14,222809.0,210809.732874,433618.732874,19663.204558,0.045347
2025-02-17,222809.0,211584.491799,434393.491799,774.758924,0.001784


In [22]:
# dfi.export(res, './tmp/private_account.png',table_conversion='matplotlib')

In [23]:
infodf = pd.DataFrame({'label': ['Update date', 'Backtest start date'],
                    'value': [TODAY, BT_START]})

dfi.export(infodf, f'./tmp/summary_info.png',table_conversion='matplotlib')